In [156]:
import pandas as pd
import pickle
from collections import Counter
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Lucas
[nltk_data]     Alponti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [136]:
with open(r'df-processado.pickle', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [137]:
df = df[['Assunto', 'Texto tratado']]
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [138]:
v = df['Texto tratado'].value_counts()
repetidos = df[df['Texto tratado'].isin(v.index[v.gt(1)])]

In [139]:
df.drop(list(repetidos.index), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [140]:
df['Tokenizadas'] = [str(t).split() for t in df['Texto tratado']]

In [141]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [142]:
dict_stopwords = Counter(stopwords)

In [143]:
for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if dict_stopwords[t]==0 and len(t)>2]

In [144]:
df['Tokenizadas']

0        [recursos, extraordinários, apelaç, comarca, s...
1        [imposto, propriedade, predial, territorial, u...
2        [acórdão, tributo, imposto, predial, territori...
3        [acórdão, mandado, segurança, iptu, progressiv...
4        [função, social, imóvel, tornar, tributo, natu...
                               ...                        
81407    [indenizatória, contexto, probatório, enseja, ...
81408    [acórdão, processo, ação, indenizatóría, afast...
81409    [penhora, direito, preferência, anterioridade,...
81410    [agravo, instrumento, ação, indenização, fase,...
81411    [vistos, relatados, discutidos, autos, apelaçã...
Name: Tokenizadas, Length: 81412, dtype: object

In [145]:
lista_palavras = []
for row in df['Texto tratado']:
    for word in str(row).split():
        lista_palavras.append(word)

counts = Counter(lista_palavras)

In [146]:
lista_pouco_comuns = [elem for elem in counts.most_common() if elem[1]<=10]

In [147]:
100*len(lista_pouco_comuns)/len(lista_palavras)

1.5207713697611012

In [148]:
for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if counts[t]>10]

In [149]:
df

,Assunto,Texto tratado,Tokenizadas
0,Direito Tributário,recursos extraordinários ma apelaç ne ô í coma...,"[recursos, extraordinários, apelaç, comarca, s..."
1,Direito Tributário,imposto propriedade predial e territorial urba...,"[imposto, propriedade, predial, territorial, u..."
2,Direito Tributário,acórdão tributo imposto predial e territorial ...,"[acórdão, tributo, imposto, predial, territori..."
3,Direito Tributário,acórdão mandado de segurança iptu progressivid...,"[acórdão, mandado, segurança, iptu, progressiv..."
4,Direito Tributário,da função social do imóvel para se tornar um t...,"[função, social, imóvel, tornar, tributo, natu..."
...,...,...,...
81407,Direito Civil,indenizatória contexto probatório que enseja o...,"[indenizatória, contexto, probatório, enseja, ..."
81408,Direito Civil,acórdão processo ação indenizatóría afastada a...,"[acórdão, processo, ação, indenizatóría, afast..."
81409,Direito Civil,penhora direito de preferência anterioridade d...,"[penhora, direito, preferência, anterioridade,..."
81410,Direito Civil,agravo df instrumento ação de indenização em f...,"[agravo, instrumento, ação, indenização, fase,..."


In [162]:
lista_palavras2 = []
for row in df['Tokenizadas']:
    for word in row:
        lista_palavras2.append(word)

counts2 = Counter(lista_palavras2)

In [186]:
lista_palavras2_corrigir = [t for t in lista_palavras2 if re.search(r'[\^~´`]|ã$|ç$', t)]
lista_palavras2_corrigir = list(set(lista_palavras2_corrigir))

In [188]:
lista_palavras2_corrigir.remove('manhã')
lista_palavras2_corrigir.remove('mairiporã')
lista_palavras2_corrigir.remove('irmã')
lista_palavras2_corrigir.remove('guardiã')

In [191]:
dict_corrigir = Counter(lista_palavras2_corrigir)

for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if dict_corrigir[t]==0 and len(t)>2]

In [199]:
df.drop(df[df['Tokenizadas'].apply(lambda x: len(str(x))==0)].index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [200]:
with open(r'df-tokenizado.pickle', 'wb') as picklefile:
    pickle.dump(df, picklefile, protocol=pickle.HIGHEST_PROTOCOL)